# California Well Completion Reports Datasets

Related links:
* For the documentation about this dataset, its source, how to download, and the features of interest, please refer to our [Well Completion Reports Dataset](/doc/assets/well_completion_reports.md) documentation


In [ ]:
!pip install ..

Processing /work/milestone2_waterwells_deepnote
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for lib: filename=lib-0.1-py3-none-any.whl size=16568 sha256=8f087a1f3d4fbc488f1e0aca15ed15aeb241fae4b3c3bca4d4ad6608ad9636d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbj6syso/wheels/98/27/77/03f18e1613edc23455313ea7117915ebf8088f55be7a77d968
Successfully built lib
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
from lib.wellcompletionreports import WellCompletionReportsDataset

/shared-libs/python3.9/py/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import pygeos

import altair as alt



In [ ]:
# import sys
# import os

# # Note once library path is set this will not be require: REMOVE
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path+"\\lib")
# from lib.wellcompletionreports import WellCompletionReportsDataset

In [ ]:
#This will be stored in the class
wcr_instance = WellCompletionReportsDataset()


/root/venv/lib/python3.9/site-packages/lib/wellcompletionreports.py:30: DtypeWarning: Columns (1,6,31) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(input_datafile)


In [ ]:
wcr_instance.clean_well_completion_reports()

,LATITUDE,LONGITUDE,TOWNSHIP,RANGE,SECTION,WELLLOCATION,CITY,COUNTY,BOTTOMOFPERFORATEDINTERVAL,TOPOFPERFORATEDINTERVAL,...,WCRNUMBER,TOTALDRILLDEPTH,TOTALCOMPLETEDDEPTH,DATEWORKENDED,WELLYIELD,WELLYIELDUNITOFMEASURE,TOTALCOMPLETEDDEPTH_CORRECTED,DATEWORKENDED_CORRECTED,YEARWORKENDED,MONTHWORKENDED
4,37.431500,-122.080000,06S,02W,09,22148 Eden Canyon RD,Castro Valley,Alameda,840.0,600.0,...,WCR2019-015599,840.0,840.0,2019-10-22,1.0,GPM,840.0,2019-10-22,2019.0,10.0
67,37.644024,-121.893386,03S,01E,29,0 CASTLEWOOD DR,PLEASANTON,Alameda,190.0,90.0,...,WCR2020-000099,230.0,210.0,2019-11-15,100.0,GPM,210.0,2019-11-15,2019.0,11.0
90,37.658240,-121.850860,03S,01E,22,NaN,NaN,Alameda,220.0,120.0,...,WCR0172727,NaN,240.0,NaT,100.0,GPM,240.0,NaT,NaN,NaN
130,37.687320,-121.850730,03S,01E,10,EL CHARRO ROAD-NW COMER OF LAKE H,PLEASANTON,Alameda,100.0,0.0,...,WCR2014-016112,NaN,205.0,2014-05-21,NaN,NaN,205.0,2014-05-21,2014.0,5.0
196,37.542150,-121.996580,04S,01W,32,NaN,NaN,Alameda,380.0,351.0,...,WCR0079408,NaN,396.0,1954-04-01,58.0,GPM,396.0,1954-04-01,1954.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042989,37.614012,-121.817013,04S,01E,01,7010 VALLECITOS ROAD,SUNOL,NaN,297.0,145.0,...,WCR2011-011692,NaN,320.0,2011-08-02,NaN,NaN,320.0,2011-08-02,2011.0,8.0
1042993,38.227086,-121.466892,04N,05E,06,26501 N. VAIL ROAD,NaN,NaN,165.0,105.0,...,WCR2015-014442,NaN,165.0,2015-10-21,NaN,NaN,165.0,2015-10-21,2015.0,10.0
1043027,38.751944,-120.496667,11N,13E,35,PARK CREEK,NaN,NaN,220.0,200.0,...,WCR2014-014504,NaN,220.0,2014-08-30,25.0,GPM,220.0,2014-08-30,2014.0,8.0
1043028,38.848056,-121.497778,12N,04E,35,XXX,XXX,NaN,160.0,1.0,...,WCR2015-013869,NaN,250.0,2015-04-21,30.0,GPM,250.0,2015-04-21,2015.0,4.0


In [ ]:
wcr_instance.wellcompletion_subset_df

,LATITUDE,LONGITUDE,TOWNSHIP,RANGE,SECTION,WELLLOCATION,CITY,COUNTY,BOTTOMOFPERFORATEDINTERVAL,TOPOFPERFORATEDINTERVAL,...,WCRNUMBER,TOTALDRILLDEPTH,TOTALCOMPLETEDDEPTH,DATEWORKENDED,WELLYIELD,WELLYIELDUNITOFMEASURE,TOTALCOMPLETEDDEPTH_CORRECTED,DATEWORKENDED_CORRECTED,YEARWORKENDED,MONTHWORKENDED
4,37.431500,-122.080000,06S,02W,09,22148 Eden Canyon RD,Castro Valley,Alameda,840.0,600.0,...,WCR2019-015599,840.0,840.0,2019-10-22,1.0,GPM,840.0,2019-10-22,2019.0,10.0
67,37.644024,-121.893386,03S,01E,29,0 CASTLEWOOD DR,PLEASANTON,Alameda,190.0,90.0,...,WCR2020-000099,230.0,210.0,2019-11-15,100.0,GPM,210.0,2019-11-15,2019.0,11.0
90,37.658240,-121.850860,03S,01E,22,NaN,NaN,Alameda,220.0,120.0,...,WCR0172727,NaN,240.0,NaT,100.0,GPM,240.0,NaT,NaN,NaN
130,37.687320,-121.850730,03S,01E,10,EL CHARRO ROAD-NW COMER OF LAKE H,PLEASANTON,Alameda,100.0,0.0,...,WCR2014-016112,NaN,205.0,2014-05-21,NaN,NaN,205.0,2014-05-21,2014.0,5.0
196,37.542150,-121.996580,04S,01W,32,NaN,NaN,Alameda,380.0,351.0,...,WCR0079408,NaN,396.0,1954-04-01,58.0,GPM,396.0,1954-04-01,1954.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042989,37.614012,-121.817013,04S,01E,01,7010 VALLECITOS ROAD,SUNOL,NaN,297.0,145.0,...,WCR2011-011692,NaN,320.0,2011-08-02,NaN,NaN,320.0,2011-08-02,2011.0,8.0
1042993,38.227086,-121.466892,04N,05E,06,26501 N. VAIL ROAD,NaN,NaN,165.0,105.0,...,WCR2015-014442,NaN,165.0,2015-10-21,NaN,NaN,165.0,2015-10-21,2015.0,10.0
1043027,38.751944,-120.496667,11N,13E,35,PARK CREEK,NaN,NaN,220.0,200.0,...,WCR2014-014504,NaN,220.0,2014-08-30,25.0,GPM,220.0,2014-08-30,2014.0,8.0
1043028,38.848056,-121.497778,12N,04E,35,XXX,XXX,NaN,160.0,1.0,...,WCR2015-013869,NaN,250.0,2015-04-21,30.0,GPM,250.0,2015-04-21,2015.0,4.0


In [ ]:
wcr_instance.draw_mising_data_chart()

alt.LayerChart(...)

In [ ]:
wcr_instance.merge_data_plss()

In [ ]:
wcr_instance.wellcompletion_subset_plss.sample(5)

,LATITUDE,LONGITUDE,TOWNSHIP,RANGE,SECTION,WELLLOCATION,CITY,COUNTY,BOTTOMOFPERFORATEDINTERVAL,TOPOFPERFORATEDINTERVAL,...,geometry,index_right,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange
667150,37.728870,-121.485700,02S,04E,25,NaN,TRACY,San Joaquin,NaN,NaN,...,POINT (-121.48570 37.72887),1081.0,12457.0,T02S,R04E,MDM,BLM,25.0,MDM-T02S-R04E-25,T02S R04E
919769,37.427540,-120.931920,06S,09E,12,FAITH HOME RD,NaN,Stanislaus,91.0,71.0,...,POINT (-120.93192 37.42754),3185.0,23655.0,T06S,R09E,MDM,BLM,12.0,MDM-T06S-R09E-12,T06S R09E
669270,38.099167,-121.125833,03N,08E,20,12975 N TULLY RD,LODI,San Joaquin,380.0,280.0,...,POINT (-121.12583 38.09917),1563.0,14230.0,T03N,R08E,MDM,BLM,20.0,MDM-T03N-R08E-20,T03N R08E
175916,36.568770,-119.890190,16S,19E,03,5161 W FLORAL AVE,FRESNO,Fresno,NaN,NaN,...,POINT (-119.89019 36.56877),7637.0,53993.0,T16S,R19E,MDM,BLM,3.0,MDM-T16S-R19E-3,T16S R19E
74764,38.145000,-120.881389,03N,10E,04,125 LOW LAKEVIEW ROAD,BURSON,Calaveras,577.0,430.0,...,POINT (-120.88139 38.14500),53.0,3724.0,T03N,R10E,MDM,BLM,4.0,MDM-T03N-R10E-4,T03N R10E


#### The below are the functions that were tested in the notebook before converting them into classes

In [ ]:
def get_plss_and_wellcompletion():
    #initialize an instance
    wcr_instance = WellCompletionReportsDataset()

    # load the plss shapefile (these only include TRS areas that are within the San Joaquin subbasin)
    SJ_subbasin_plss = wcr_instance.map_df
    # aggregate by TownshipRange
    SJ_subbasin_plss_range = SJ_subbasin_plss.dissolve(by='TownshipRange').reset_index()

    wellcompletion_df = wcr_instance.data_df
    wellcompletion_df = wellcompletion_df.iloc[:, 1:].copy()
    
    return wcr_instance, wellcompletion_df



In [ ]:
#SJ_subbasin_plss.explore()

In [ ]:
#SJ_subbasin_plss_range.explore()

### Well Completion data

[Source](https://data.cnra.ca.gov/dataset/well-completion-reports)

In [ ]:
def draw_mising_data_chart(df):
    percent_missing = df.isnull().sum() / len(wellcompletion_df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', ascending = False, inplace=True)

    sort_list = list(missing_value_df['column_name'])
    chart = alt.Chart(missing_value_df
                     ).mark_bar(
                        ).encode(
                    y =alt.Y("sum(percent_missing)", stack="normalize", axis=alt.Axis(format='%')),
                    x = alt.X('column_name:N', sort=sort_list),
                    color=alt.value("orange"),
                    tooltip = ['column_name', 'percent_missing']
                    )
    
    
    text = chart.transform_calculate(
        position = 'datum.percent_missing + 0.05 * datum.percent_missing / abs(datum.percent_missing)'
    ).mark_text(
        align='center', 
        fontSize=10,
        color='black'
    ).encode(
        y='position:Q',
        text=alt.Text('percent_missing:Q', format='.0%'),
    )

    
    return chart + text


In [ ]:
#Draw a chart showing percent missing values for the features we are interested in
draw_mising_data_chart(wcr_instance.data_df[["DECIMALLATITUDE", "DECIMALLONGITUDE", "TOWNSHIP", "RANGE", "SECTION", "WELLLOCATION", "CITY", "COUNTYNAME", 
                                              "BOTTOMOFPERFORATEDINTERVAL", "TOPOFPERFORATEDINTERVAL", "GROUNDSURFACEELEVATION", "STATICWATERLEVEL", 
                                              "RECORDTYPE",  "PLANNEDUSEFORMERUSE", "WCRNUMBER", "TOTALDRILLDEPTH", 
                                              "TOTALCOMPLETEDDEPTH", "DATEWORKENDED", 'WELLYIELD', 'WELLYIELDUNITOFMEASURE']])

In [ ]:
def clean_well_completion_reports():
    wellcompletion_df = wcr_instance.data_df

    #There are latitudes and longitudes that are corrupt : 37/41/11.82/
    wellcompletion_df = wellcompletion_df[~wellcompletion_df.DECIMALLATITUDE.str.contains(r"/", na=False)].copy()
    wellcompletion_df = wellcompletion_df[~wellcompletion_df.DECIMALLONGITUDE.str.contains(r"/", na=False)].copy()

    wellcompletion_df['DECIMALLATITUDE'] = wellcompletion_df.DECIMALLATITUDE.astype('float')
    wellcompletion_df['DECIMALLONGITUDE'] = wellcompletion_df.DECIMALLONGITUDE.astype('float')


    #Correct incorrectly signed logitude and latiude Example :   120.54483 Longitude
    wellcompletion_df['DECIMALLONGITUDE'] = np.where(wellcompletion_df['DECIMALLONGITUDE'] > 0,
                                                    -wellcompletion_df['DECIMALLONGITUDE'],
                                                    wellcompletion_df['DECIMALLONGITUDE'])

    wellcompletion_df['DECIMALLATITUDE'] = np.where(wellcompletion_df['DECIMALLATITUDE'] < 0,
                                                    -wellcompletion_df['DECIMALLATITUDE'],
                                                    wellcompletion_df['DECIMALLATITUDE'])

    #About 5% of the dataframe has eith latitude or longitude missing, we drop these
    wellcompletion_df = wellcompletion_df.dropna(subset=['DECIMALLATITUDE', 'DECIMALLONGITUDE']).copy()

    # Pick data of interest
    wellcompletion_subset_df = wellcompletion_df[["DECIMALLATITUDE", "DECIMALLONGITUDE", "TOWNSHIP", "RANGE", "SECTION", "WELLLOCATION", "CITY", "COUNTYNAME", 
                                                  "BOTTOMOFPERFORATEDINTERVAL", "TOPOFPERFORATEDINTERVAL", "GROUNDSURFACEELEVATION", "STATICWATERLEVEL", 
                                                  "RECORDTYPE",  "PLANNEDUSEFORMERUSE", "WCRNUMBER", "TOTALDRILLDEPTH", 
                                                  "TOTALCOMPLETEDDEPTH", "DATEWORKENDED", 'WELLYIELD', 'WELLYIELDUNITOFMEASURE']].copy()



    #len(wellcompletion_subset_df[(wellcompletion_subset_df['LATITUDE'].isnull()) | (wellcompletion_subset_df['LATITUDE'].isnull())])/  len(wellcompletion_subset_df)
    #.05

    # rename columns
    wellcompletion_subset_df.rename(columns={"DECIMALLATITUDE" : "LATITUDE", 
                                            "DECIMALLONGITUDE" : "LONGITUDE", 
                                            "PLANNEDUSEFORMERUSE": "USE" ,       
                                            "COUNTYNAME" : "COUNTY", 
                                           }, inplace=True)

    # filter to only include new well completion since we predict on this
    wellcompletion_subset_df = wellcompletion_subset_df.loc[wellcompletion_subset_df['RECORDTYPE'] == 'WellCompletion/New/Production or Monitoring/NA']

    # filter to only include agriculture, domestic, or public wells
    #Data issues Agriculture is also denoted by "AG"
    wellcompletion_subset_df['USE'] = wellcompletion_subset_df['USE'].fillna("")
    wellcompletion_subset_df['USE'] = wellcompletion_subset_df['USE'].str.lower()
    wellcompletion_subset_df['USE'] = (
                                        np.where(wellcompletion_subset_df['USE'].str.contains("agri|irrigation"),
                                                 "Agriculture",
                                                 np.where(wellcompletion_subset_df['USE'].str.contains("domestic"),
                                                         "Domestic",
                                                          np.where(wellcompletion_subset_df['USE'].str.contains("indus|commerc"),
                                                          "Industrial",
                                                          np.where(wellcompletion_subset_df['USE'].str.contains("public"),
                                                                  "Public",
                                                                  "Other")
                                                         )
                                                )
                                        ))
    wellcompletion_subset_df = wellcompletion_subset_df[wellcompletion_subset_df["USE"].isin(["Agriculture","Domestic","Public", "Industrial"])]

    wellcompletion_subset_df['TOTALCOMPLETEDDEPTH'] = pd.to_numeric(wellcompletion_subset_df['TOTALCOMPLETEDDEPTH'], errors="coerce")

    # removes depth data that are less than 20'
    wellcompletion_subset_df['TOTALCOMPLETEDDEPTH_CORRECTED'] = wellcompletion_subset_df['TOTALCOMPLETEDDEPTH'].apply(lambda x: x if x >= 20 else np.nan)
    # convert date work ended to datetime and filter to only include completed dates that are possible (not a future date) 
    wellcompletion_subset_df['DATEWORKENDED'] = pd.to_datetime(wellcompletion_subset_df['DATEWORKENDED'], errors='coerce')
    # convert date work ended to datetime and filter to only include completed dates that are possible (not a future date) 
    wellcompletion_subset_df['DATEWORKENDED'] = pd.to_datetime(wellcompletion_subset_df['DATEWORKENDED'], errors='coerce')
    wellcompletion_subset_df['DATEWORKENDED_CORRECTED'] = wellcompletion_subset_df['DATEWORKENDED'].apply(lambda x: x if x < datetime.now() else np.nan)
    # create simple year and month columns
    wellcompletion_subset_df['YEARWORKENDED'] = pd.DatetimeIndex(wellcompletion_subset_df['DATEWORKENDED_CORRECTED']).year
    wellcompletion_subset_df['MONTHWORKENDED'] = pd.DatetimeIndex(wellcompletion_subset_df['DATEWORKENDED_CORRECTED']).month
    
    return wellcompletion_subset_df

In [ ]:
def merge_data_plss():
    df = self.wellcompletion_subset_df
    # create wells geodataframe
    wellcompletion_subset_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))
    #Set the coordinate reference system (the projection that denote the axis for the points)
    wellcompletion_subset_gdf = wellcompletion_subset_gdf.set_crs('epsg:4326')

    # spatial join based on geometry
    wellcompletion_subset_plss = wellcompletion_subset_gdf.sjoin(SJ_subbasin_plss, how="left")

    # drop the ones that aren't in the san joaquin valley basin
    wellcompletion_subset_plss = wellcompletion_subset_plss.dropna(subset=['MTRS'])
    
    return wellcompletion_subset_plss

In [ ]:
#len(wellcompletion_subset_df)/ len(wcr_instance.data_df) # Cleaned dataset =  0.43264636176071297  of original

In [ ]:
wellcompletion_subset_plss.shape #(105987, 34)

In [ ]:
len(wellcompletion_subset_plss)/ len(wcr_instance.data_df)# San Joaquin dataset =  0.10161433206267881  of original

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b042e2da-6536-449d-95b8-d85fa08825de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>